In [1]:
import os
import shutil
import time
from typing import Optional

import pandas as pd
import numpy as np
import requests
import zipfile
import gc
import tracemalloc

In [2]:
# Configuration
YEARS = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
DATA_DIR = os.path.join(os.getcwd(), "..", "test", "data_raw")
PROCESSED_DATA_DIR = os.path.join(os.getcwd(), "..", "test", "data_processed")
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

BASE_URL = "https://www.cdc.gov/brfss/annual_data/{year}/files/LLCP{year}XPT.zip"

In [3]:
# Variable mapping across years
VAR_MAP = {
	"HEALTH_STATUS":{2015:"_RFHLTH",2016:"_RFHLTH",2017:"_RFHLTH",2018:"_RFHLTH",2019:"_RFHLTH",2020:"_RFHLTH",2021:"_RFHLTH",2022:"_RFHLTH",2023:"_RFHLTH",2024:"_RFHLTH"},
	"PHYSICAL_HEALTH_STATUS":{2015:"PHYSHLTH",2016:"_PHYS14D",2017:"_PHYS14D",2018:"_PHYS14D",2019:"_PHYS14D",2020:"_PHYS14D",2021:"_PHYS14D",2022:"_PHYS14D",2023:"_PHYS14D",2024:"_PHYS14D"},
	"MENTAL_HEALTH_STATUS":{2015:"MENTHLTH",2016:"_MENT14D",2017:"_MENT14D",2018:"_MENT14D",2019:"_MENT14D",2020:"_MENT14D",2021:"_MENT14D",2022:"_MENT14D",2023:"_MENT14D",2024:"_MENT14D"},
	"EXERCISE":{2015:"_TOTINDA",2016:"_TOTINDA",2017:"_TOTINDA",2018:"_TOTINDA",2019:"_TOTINDA",2020:"_TOTINDA",2021:"_TOTINDA",2022:"_TOTINDA",2023:"_TOTINDA",2024:"_TOTINDA"},
	
	"HEALTH_CARE_COVERAGE":{2015:"_HCVU651",2016:"_HCVU651",2017:"_HCVU651",2018:"_HCVU651",2019:"_HCVU651",2020:"_HCVU651",2021:"_HCVU652",2022:"_HCVU652",2023:"_HCVU653",2024:"_HCVU654"},
	
	"SMOKER":{2015:"_SMOKER3",2016:"_SMOKER3",2017:"_SMOKER3",2018:"_SMOKER3",2019:"_SMOKER3",2020:"_SMOKER3",2021:"_SMOKER3",2022:"_SMOKER3",2023:"_SMOKER3",2024:"_SMOKER3"},
	"DRINKER":{2015:"DRNKANY5",2016:"DRNKANY5",2017:"DRNKANY5",2018:"DRNKANY5",2019:"DRNKANY5",2020:"DRNKANY5",2021:"DRNKANY5",2022:"DRNKANY6",2023:"DRNKANY6",2024:"DRNKANY6"},
	"SOCIAL_DRINKER":{2015:"_RFBING5",2016:"_RFBING5",2017:"_RFBING5",2018:"_RFBING5",2019:"_RFBING5",2020:"_RFBING5",2021:"_RFBING5",2022:"_RFBING6",2023:"_RFBING6",2024:"_RFBING6"},
    "HEAVY_ALCOHOL_CONSUMPTION":{2015:"_RFDRHV5",2016:"_RFDRHV5",2017:"_RFDRHV5",2018:"_RFDRHV6",2019:"_RFDRHV7",2020:"_RFDRHV7",2021:"_RFDRHV7",2022:"_RFDRHV8",2023:"_RFDRHV8",2024:"_RFDRHV9"},
	
    "HEART_ATTACK":{2015:"_MICHD",2016:"_MICHD",2017:"_MICHD",2018:"_MICHD",2019:"_MICHD",2020:"_MICHD",2021:"_MICHD",2022:"_MICHD",2023:"_MICHD",2024:"_MICHD"},
	"STROKE":{2015:"CVDSTRK3",2016:"CVDSTRK3",2017:"CVDSTRK3",2018:"CVDSTRK3",2019:"CVDSTRK3",2020:"CVDSTRK3",2021:"CVDSTRK3",2022:"CVDSTRK3",2023:"CVDSTRK3",2024:"CVDSTRK3"},
    "DIABETES":{2015:"DIABETE3",2016:"DIABETE3",2017:"DIABETE3",2018:"DIABETE3",2019:"DIABETE4",2020:"DIABETE4",2021:"DIABETE4",2022:"DIABETE4",2023:"DIABETE4",2024:"DIABETE4"},
	"ARTHRITIS":{2015:"_DRDXAR1",2016:"_DRDXAR1",2017:"_DRDXAR1",2018:"_DRDXAR1",2019:"_DRDXAR2",2020:"_DRDXAR2",2021:"_DRDXAR3",2022:"_DRDXAR2",2023:"_DRDXAR2",2024:"_DRDXAR2"},
	
    "MARITAL_STATUS":{2015:"MARITAL",2016:"MARITAL",2017:"MARITAL",2018:"MARITAL",2019:"MARITAL",2020:"MARITAL",2021:"MARITAL",2022:"MARITAL",2023:"MARITAL",2024:"MARITAL"},
    "EMPLOYMENT":{2015:"EMPLOY1",2016:"EMPLOY1",2017:"EMPLOY1",2018:"EMPLOY1",2019:"EMPLOY1",2020:"EMPLOY1",2021:"EMPLOY1",2022:"EMPLOY1",2023:"EMPLOY1",2024:"EMPLOY1"},
    "SEX":{2015:"SEX",2016:"SEX",2017:"SEX",2018:"SEX1",2019:"_SEX",2020:"_SEX",2021:"_SEX",2022:"_SEX",2023:"_SEX",2024:"_SEX"},
	"AGE_CATEGORIES":{2015:"_AGEG5YR",2016:"_AGEG5YR",2017:"_AGEG5YR",2018:"_AGEG5YR",2019:"_AGEG5YR",2020:"_AGEG5YR",2021:"_AGEG5YR",2022:"_AGEG5YR",2023:"_AGEG5YR",2024:"_AGEG5YR"},
    "BMICAT":{2015:"_BMI5CAT",2016:"_BMI5CAT",2017:"_BMI5CAT",2018:"_BMI5CAT",2019:"_BMI5CAT",2020:"_BMI5CAT",2021:"_BMI5CAT",2022:"_BMI5CAT",2023:"_BMI5CAT",2024:"_BMI5CAT"},
	"EDUCATION_LEVEL":{2015:"_EDUCAG",2016:"_EDUCAG",2017:"_EDUCAG",2018:"_EDUCAG",2019:"_EDUCAG",2020:"_EDUCAG",2021:"_EDUCAG",2022:"_EDUCAG",2023:"_EDUCAG",2024:"_EDUCAG"},
	"INCOME":{2015:"_INCOMG",2016:"_INCOMG",2017:"_INCOMG",2018:"_INCOMG",2019:"_INCOMG",2020:"_INCOMG",2021:"_INCOMG1",2022:"_INCOMG1",2023:"_INCOMG1",2024:"_INCOMG1"},
}

In [4]:
# Value mapping across years
VALUE_MAP = {
    "HEALTH_STATUS": {
        2015: {1:"Good",2:"Poor",9:"Refused"},
        2016: {1:"Good",2:"Poor",9:"Refused"},
        2017: {1:"Good",2:"Poor",9:"Refused"},
        2018: {1:"Good",2:"Poor",9:"Refused"},
        2019: {1:"Good",2:"Poor",9:"Refused"},
        2020: {1:"Good",2:"Poor",9:"Refused"},
        2021: {1:"Good",2:"Poor",9:"Refused"},
        2022: {1:"Good",2:"Poor",9:"Refused"},
        2023: {1:"Good",2:"Poor",9:"Refused"},
        2024: {1:"Good",2:"Poor",9:"Refused"}
    },
    "PHYSICAL_HEALTH_STATUS": {
        2015: {88:"Zero days",1:"1-13 days",2:"1-13 days",3:"1-13 days",4:"1-13 days",5:"1-13 days",6:"1-13 days",7:"1-13 days",8:"1-13 days",9:"1-13 days",10:"1-13 days",11:"1-13 days",12:"1-13 days",13:"14+ days",14:"14+ days",15:"14+ days",16:"14+ days",17:"14+ days",18:"14+ days",19:"14+ days",20:"14+ days",21:"14+ days",22:"14+ days",23:"14+ days",24:"14+ days",25:"14+ days",26:"14+ days",27:"14+ days",28:"14+ days",29:"14+ days",30:"14+ days",31:"14+ days",32:"14+ days",99:"Refused"},
        2016: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
        2017: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
        2018: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
        2019: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
        2020: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
        2021: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
        2022: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
        2023: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
        2024: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"}
    },
    "MENTAL_HEALTH_STATUS": {
        2015: {88:"Zero days",1:"1-13 days",2:"1-13 days",3:"1-13 days",4:"1-13 days",5:"1-13 days",6:"1-13 days",7:"1-13 days",8:"1-13 days",9:"1-13 days",10:"1-13 days",11:"1-13 days",12:"1-13 days",13:"14+ days",14:"14+ days",15:"14+ days",16:"14+ days",17:"14+ days",18:"14+ days",19:"14+ days",20:"14+ days",21:"14+ days",22:"14+ days",23:"14+ days",24:"14+ days",25:"14+ days",26:"14+ days",27:"14+ days",28:"14+ days",29:"14+ days",30:"14+ days",31:"14+ days",32:"14+ days",99:"Refused"},
        2016: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
        2017: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
        2018: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
        2019: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
        2020: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
        2021: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
        2022: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
        2023: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
        2024: {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"}
    },
    "EXERCISE": {
        2015: {1:"Yes",2:"No",9:"Refused"},
        2016: {1:"Yes",2:"No",9:"Refused"},
        2017: {1:"Yes",2:"No",9:"Refused"},
        2018: {1:"Yes",2:"No",9:"Refused"},
        2019: {1:"Yes",2:"No",9:"Refused"},
        2020: {1:"Yes",2:"No",9:"Refused"},
        2021: {1:"Yes",2:"No",9:"Refused"},
        2022: {1:"Yes",2:"No",9:"Refused"},
        2023: {1:"Yes",2:"No",9:"Refused"},
        2024: {1:"Yes",2:"No",9:"Refused"}
    },
    "HEALTH_CARE_COVERAGE": {
        2015: {1:"Yes",2:"No",9:"Refused"},
        2016: {1:"Yes",2:"No",9:"Refused"},
        2017: {1:"Yes",2:"No",9:"Refused"},
        2018: {1:"Yes",2:"No",9:"Refused"},
        2019: {1:"Yes",2:"No",9:"Refused"},
        2020: {1:"Yes",2:"No",9:"Refused"},
        2021: {1:"Yes",2:"No",9:"Refused"},
        2022: {1:"Yes",2:"No",9:"Refused"},
        2023: {1:"Yes",2:"No",9:"Refused"},
        2024: {1:"Yes",2:"No",9:"Refused"}
    },
	"SMOKER": {
        2015: {1:"Everyday",2:"Somedays",3:"Former",4:"Never",9:"Refused"},
        2016: {1:"Everyday",2:"Somedays",3:"Former",4:"Never",9:"Refused"},
        2017: {1:"Everyday",2:"Somedays",3:"Former",4:"Never",9:"Refused"},
        2018: {1:"Everyday",2:"Somedays",3:"Former",4:"Never",9:"Refused"},
        2019: {1:"Everyday",2:"Somedays",3:"Former",4:"Never",9:"Refused"},
		2020: {1:"Everyday",2:"Somedays",3:"Former",4:"Never",9:"Refused"},
		2021: {1:"Everyday",2:"Somedays",3:"Former",4:"Never",9:"Refused"},
		2022: {1:"Everyday",2:"Somedays",3:"Former",4:"Never",9:"Refused"},
		2023: {1:"Everyday",2:"Somedays",3:"Former",4:"Never",9:"Refused"},
		2024: {1:"Everyday",2:"Somedays",3:"Former",4:"Never",9:"Refused"}
	},
	"DRINKER": {
        2015: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
        2016: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
        2017: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
        2018: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
        2019: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
		2020: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
		2021: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
		2022: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
		2023: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
		2024: {1:"Yes",2:"No",7:"Not sure",9:"Refused"}
	},
	"SOCIAL_DRINKER": {
		2015: {1:"No",2:"Yes",9:"Refused"},
        2016: {1:"No",2:"Yes",9:"Refused"},
        2017: {1:"No",2:"Yes",9:"Refused"},
        2018: {1:"No",2:"Yes",9:"Refused"},
        2019: {1:"No",2:"Yes",9:"Refused"},
        2020: {1:"No",2:"Yes",9:"Refused"},
		2021: {1:"No",2:"Yes",9:"Refused"},
		2022: {1:"No",2:"Yes",9:"Refused"},
		2023: {1:"No",2:"Yes",9:"Refused"},
		2024: {1:"No",2:"Yes",9:"Refused"}
	},
	"HEAVY_ALCOHOL_CONSUMPTION": {
        2015: {1:"No",2:"Yes",9:"Refused"},
        2016: {1:"No",2:"Yes",9:"Refused"},
        2017: {1:"No",2:"Yes",9:"Refused"},
        2018: {1:"No",2:"Yes",9:"Refused"},
        2019: {1:"No",2:"Yes",9:"Refused"},
		2020: {1:"No",2:"Yes",9:"Refused"},
		2021: {1:"No",2:"Yes",9:"Refused"},
		2022: {1:"No",2:"Yes",9:"Refused"},
		2023: {1:"No",2:"Yes",9:"Refused"},
		2024: {1:"No",2:"Yes",9:"Refused"}		
	},
	"HEART_ATTACK": {
        2015: {1:"Yes",2:"No"},
        2016: {1:"Yes",2:"No"},
        2017: {1:"Yes",2:"No"},
        2018: {1:"Yes",2:"No"},
        2019: {1:"Yes",2:"No"},
		2020: {1:"Yes",2:"No"},
		2021: {1:"Yes",2:"No"},
		2022: {1:"Yes",2:"No"},
		2023: {1:"Yes",2:"No"},
		2024: {1:"Yes",2:"No"}
	},
	"STROKE": {
        2015: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
        2016: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
        2017: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
        2018: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
        2019: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
		2020: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
		2021: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
		2022: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
		2023: {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
		2024: {1:"Yes",2:"No",7:"Not sure",9:"Refused"}
	},
	"DIABETES": {
        2015: {1:"Diabetes",2:"Gestational",3:"No",4:"PreDiabetes",7:"Not sure",9:"Refused"},
        2016: {1:"Diabetes",2:"Gestational",3:"No",4:"PreDiabetes",7:"Not sure",9:"Refused"},
        2017: {1:"Diabetes",2:"Gestational",3:"No",4:"PreDiabetes",7:"Not sure",9:"Refused"},
        2018: {1:"Diabetes",2:"Gestational",3:"No",4:"PreDiabetes",7:"Not sure",9:"Refused"},
        2019: {1:"Diabetes",2:"Gestational",3:"No",4:"PreDiabetes",7:"Not sure",9:"Refused"},
		2020: {1:"Diabetes",2:"Gestational",3:"No",4:"PreDiabetes",7:"Not sure",9:"Refused"},
		2021: {1:"Diabetes",2:"Gestational",3:"No",4:"PreDiabetes",7:"Not sure",9:"Refused"},
		2022: {1:"Diabetes",2:"Gestational",3:"No",4:"PreDiabetes",7:"Not sure",9:"Refused"},
		2023: {1:"Diabetes",2:"Gestational",3:"No",4:"PreDiabetes",7:"Not sure",9:"Refused"},
		2024: {1:"Diabetes",2:"Gestational",3:"No",4:"PreDiabetes",7:"Not sure",9:"Refused"}
	},
	"ARTHRITIS": {
        2015: {1:"Yes",2:"No"},
        2016: {1:"Yes",2:"No"},
        2017: {1:"Yes",2:"No"},
        2018: {1:"Yes",2:"No"},
        2019: {1:"Yes",2:"No"},
        2020: {1:"Yes",2:"No"},
        2021: {1:"Yes",2:"No"},
        2022: {1:"Yes",2:"No"},
        2023: {1:"Yes",2:"No"},
        2024: {1:"Yes",2:"No"}
	},
	"MARITAL_STATUS": {
        2015: {1:"Married",2:"Divorced",3:"Widowed",4:"Separated",5:"Single",6:"Partner",9:"Refused"},
        2016: {1:"Married",2:"Divorced",3:"Widowed",4:"Separated",5:"Single",6:"Partner",9:"Refused"},
        2017: {1:"Married",2:"Divorced",3:"Widowed",4:"Separated",5:"Single",6:"Partner",9:"Refused"},
        2018: {1:"Married",2:"Divorced",3:"Widowed",4:"Separated",5:"Single",6:"Partner",9:"Refused"},
        2019: {1:"Married",2:"Divorced",3:"Widowed",4:"Separated",5:"Single",6:"Partner",9:"Refused"},
		2020: {1:"Married",2:"Divorced",3:"Widowed",4:"Separated",5:"Single",6:"Partner",9:"Refused"},
		2021: {1:"Married",2:"Divorced",3:"Widowed",4:"Separated",5:"Single",6:"Partner",9:"Refused"},
		2022: {1:"Married",2:"Divorced",3:"Widowed",4:"Separated",5:"Single",6:"Partner",9:"Refused"},
		2023: {1:"Married",2:"Divorced",3:"Widowed",4:"Separated",5:"Single",6:"Partner",9:"Refused"},
		2024: {1:"Married",2:"Divorced",3:"Widowed",4:"Separated",5:"Single",6:"Partner",9:"Refused"}
	},
	"EMPLOYMENT": {
        2015: {1:"Wages",2:"Self-Employed",3:"Unemployed > 1 year",4:"Unemployed < 1 year",5:"Homemaker",6:"Student",7:"Retired",8:"Unable to work",9:"Refused"},
        2016: {1:"Wages",2:"Self-Employed",3:"Unemployed > 1 year",4:"Unemployed < 1 year",5:"Homemaker",6:"Student",7:"Retired",8:"Unable to work",9:"Refused"},
        2017: {1:"Wages",2:"Self-Employed",3:"Unemployed > 1 year",4:"Unemployed < 1 year",5:"Homemaker",6:"Student",7:"Retired",8:"Unable to work",9:"Refused"},
        2018: {1:"Wages",2:"Self-Employed",3:"Unemployed > 1 year",4:"Unemployed < 1 year",5:"Homemaker",6:"Student",7:"Retired",8:"Unable to work",9:"Refused"},
        2019: {1:"Wages",2:"Self-Employed",3:"Unemployed > 1 year",4:"Unemployed < 1 year",5:"Homemaker",6:"Student",7:"Retired",8:"Unable to work",9:"Refused"},
		2020: {1:"Wages",2:"Self-Employed",3:"Unemployed > 1 year",4:"Unemployed < 1 year",5:"Homemaker",6:"Student",7:"Retired",8:"Unable to work",9:"Refused"},
		2021: {1:"Wages",2:"Self-Employed",3:"Unemployed > 1 year",4:"Unemployed < 1 year",5:"Homemaker",6:"Student",7:"Retired",8:"Unable to work",9:"Refused"},
		2022: {1:"Wages",2:"Self-Employed",3:"Unemployed > 1 year",4:"Unemployed < 1 year",5:"Homemaker",6:"Student",7:"Retired",8:"Unable to work",9:"Refused"},
		2023: {1:"Wages",2:"Self-Employed",3:"Unemployed > 1 year",4:"Unemployed < 1 year",5:"Homemaker",6:"Student",7:"Retired",8:"Unable to work",9:"Refused"},
		2024: {1:"Wages",2:"Self-Employed",3:"Unemployed > 1 year",4:"Unemployed < 1 year",5:"Homemaker",6:"Student",7:"Retired",8:"Unable to work",9:"Refused"}
	},
	"SEX": {
        2015: {1:"Male",2:"Female"},
        2016: {1:"Male",2:"Female",9:"Refused"},
        2017: {1:"Male",2:"Female",9:"Refused"},
        2018: {1:"Male",2:"Female",7:"Not Sure",9:"Refused"},
        2019: {1:"Male",2:"Female"},
		2020: {1:"Male",2:"Female"},
		2021: {1:"Male",2:"Female"},
		2022: {1:"Male",2:"Female"},
		2023: {1:"Male",2:"Female"},
		2024: {1:"Male",2:"Female"}	
	},
	"AGE_CATEGORIES": {
        2015: {1:"18-24",2:"25-29",3:"30-34",4:"35-39",5:"40-44",6:"45-49",7:"50-54",8:"55-59",9:"60-64",10:"65-69",11:"70-74",12:"75-79",13:"80+",14:"Refused"},
        2016: {1:"18-24",2:"25-29",3:"30-34",4:"35-39",5:"40-44",6:"45-49",7:"50-54",8:"55-59",9:"60-64",10:"65-69",11:"70-74",12:"75-79",13:"80+",14:"Refused"},
        2017: {1:"18-24",2:"25-29",3:"30-34",4:"35-39",5:"40-44",6:"45-49",7:"50-54",8:"55-59",9:"60-64",10:"65-69",11:"70-74",12:"75-79",13:"80+",14:"Refused"},
        2018: {1:"18-24",2:"25-29",3:"30-34",4:"35-39",5:"40-44",6:"45-49",7:"50-54",8:"55-59",9:"60-64",10:"65-69",11:"70-74",12:"75-79",13:"80+",14:"Refused"},
        2019: {1:"18-24",2:"25-29",3:"30-34",4:"35-39",5:"40-44",6:"45-49",7:"50-54",8:"55-59",9:"60-64",10:"65-69",11:"70-74",12:"75-79",13:"80+",14:"Refused"},
		2020: {1:"18-24",2:"25-29",3:"30-34",4:"35-39",5:"40-44",6:"45-49",7:"50-54",8:"55-59",9:"60-64",10:"65-69",11:"70-74",12:"75-79",13:"80+",14:"Refused"},
		2021: {1:"18-24",2:"25-29",3:"30-34",4:"35-39",5:"40-44",6:"45-49",7:"50-54",8:"55-59",9:"60-64",10:"65-69",11:"70-74",12:"75-79",13:"80+",14:"Refused"},
		2022: {1:"18-24",2:"25-29",3:"30-34",4:"35-39",5:"40-44",6:"45-49",7:"50-54",8:"55-59",9:"60-64",10:"65-69",11:"70-74",12:"75-79",13:"80+",14:"Refused"},
		2023: {1:"18-24",2:"25-29",3:"30-34",4:"35-39",5:"40-44",6:"45-49",7:"50-54",8:"55-59",9:"60-64",10:"65-69",11:"70-74",12:"75-79",13:"80+",14:"Refused"},
		2024: {1:"18-24",2:"25-29",3:"30-34",4:"35-39",5:"40-44",6:"45-49",7:"50-54",8:"55-59",9:"60-64",10:"65-69",11:"70-74",12:"75-79",13:"80+",14:"Refused"}
	},
	"BMICAT": {
        2015: {1:"Underweight",2:"Normal",3:"Overweight",4:"Obese"},
        2016: {1:"Underweight",2:"Normal",3:"Overweight",4:"Obese"},
        2017: {1:"Underweight",2:"Normal",3:"Overweight",4:"Obese"},
        2018: {1:"Underweight",2:"Normal",3:"Overweight",4:"Obese"},
        2019: {1:"Underweight",2:"Normal",3:"Overweight",4:"Obese"},
		2020: {1:"Underweight",2:"Normal",3:"Overweight",4:"Obese"},
		2021: {1:"Underweight",2:"Normal",3:"Overweight",4:"Obese"},
		2022: {1:"Underweight",2:"Normal",3:"Overweight",4:"Obese"},
		2023: {1:"Underweight",2:"Normal",3:"Overweight",4:"Obese"},
		2024: {1:"Underweight",2:"Normal",3:"Overweight",4:"Obese"},
	},
	"EDUCATION_LEVEL": {
        2015: {1:"Dropout",2:"High School Graduate",3:"Undergraduate",4:"Graduate",9:"Not sure"},
        2016: {1:"Dropout",2:"High School Graduate",3:"Undergraduate",4:"Graduate",9:"Not sure"},
        2017: {1:"Dropout",2:"High School Graduate",3:"Undergraduate",4:"Graduate",9:"Not sure"},
        2018: {1:"Dropout",2:"High School Graduate",3:"Undergraduate",4:"Graduate",9:"Not sure"},
        2019: {1:"Dropout",2:"High School Graduate",3:"Undergraduate",4:"Graduate",9:"Not sure"},
		2020: {1:"Dropout",2:"High School Graduate",3:"Undergraduate",4:"Graduate",9:"Not sure"},
		2021: {1:"Dropout",2:"High School Graduate",3:"Undergraduate",4:"Graduate",9:"Not sure"},
		2022: {1:"Dropout",2:"High School Graduate",3:"Undergraduate",4:"Graduate",9:"Not sure"},
		2023: {1:"Dropout",2:"High School Graduate",3:"Undergraduate",4:"Graduate",9:"Not sure"},
		2024: {1:"Dropout",2:"High School Graduate",3:"Undergraduate",4:"Graduate",9:"Not sure"}
	},
	"INCOME": {
        2015: {1:"< $15000",2:"$15000 - $25000",3:"$25000 - $35000",4:"$35000 - $50000",5:"> $50000",9:"Not sure"},
        2016: {1:"< $15000",2:"$15000 - $25000",3:"$25000 - $35000",4:"$35000 - $50000",5:"> $50000",9:"Not sure"},
        2017: {1:"< $15000",2:"$15000 - $25000",3:"$25000 - $35000",4:"$35000 - $50000",5:"> $50000",9:"Not sure"},
        2018: {1:"< $15000",2:"$15000 - $25000",3:"$25000 - $35000",4:"$35000 - $50000",5:"> $50000",9:"Not sure"},
        2019: {1:"< $15000",2:"$15000 - $25000",3:"$25000 - $35000",4:"$35000 - $50000",5:"> $50000",9:"Not sure"},
		2020: {1:"< $15000",2:"$15000 - $25000",3:"$25000 - $35000",4:"$35000 - $50000",5:"> $50000",9:"Not sure"},
		2021: {1:"< $15000",2:"$15000 - $25000",3:"$25000 - $35000",4:"$35000 - $50000",5:"$50000 - $100000",6:"$100000 - $200000",7:"> $200000",9:"Not sure"},
		2022: {1:"< $15000",2:"$15000 - $25000",3:"$25000 - $35000",4:"$35000 - $50000",5:"$50000 - $100000",6:"$100000 - $200000",7:"> $200000",9:"Not sure"},
		2023: {1:"< $15000",2:"$15000 - $25000",3:"$25000 - $35000",4:"$35000 - $50000",5:"$50000 - $100000",6:"$100000 - $200000",7:"> $200000",9:"Not sure"},
		2024: {1:"< $15000",2:"$15000 - $25000",3:"$25000 - $35000",4:"$35000 - $50000",5:"$50000 - $100000",6:"$100000 - $200000",7:"> $200000",9:"Not sure"}
	}
}

In [5]:
# Value mapping constant across years
VALUE_TEXT_MAP = {
    "HEALTH_STATUS": {1:"Good",2:"Poor",9:"Refused"},
    "PHYSICAL_HEALTH_STATUS": {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
    "MENTAL_HEALTH_STATUS": {1:"Zero days",2:"1-13 days",3:"14+ days",9:"Refused"},
    "EXERCISE": {1:"Yes",2:"No",9:"Refused"},
    "HEALTH_CARE_COVERAGE": {1:"Yes",2:"No",9:"Refused"},
    "SMOKER": {1:"Everyday",2:"Somedays",3:"Former",4:"Never",9:"Refused"},
    "DRINKER": {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
    "SOCIAL_DRINKER": {1:"No",2:"Yes",9:"Refused"},
    "HEAVY_ALCOHOL_CONSUMPTION": {1:"No",2:"Yes",9:"Refused"},
    "HEART_ATTACK": {1:"Yes",2:"No"},
    "STROKE": {1:"Yes",2:"No",7:"Not sure",9:"Refused"},
    "DIABETES": {1:"Diabetes",2:"Gestational",3:"No",4:"PreDiabetes",7:"Not sure",9:"Refused"},
    "ARTHRITIS": {1:"Yes",2:"No"},
    "MARITAL_STATUS": {1:"Married",2:"Divorced",3:"Widowed",4:"Separated",5:"Single",6:"Partner",9:"Refused"},
    "EMPLOYMENT": {1:"Wages",2:"Self-Employed",3:"Unemployed > 1 year",4:"Unemployed < 1 year",5:"Homemaker",6:"Student",7:"Retired",8:"Unable to work",9:"Refused"},
    "SEX": {1:"Male",2:"Female",7:"Not Sure",9:"Refused"},
    "AGE_CATEGORIES": {1:"18-24",2:"25-29",3:"30-34",4:"35-39",5:"40-44",6:"45-49",7:"50-54",8:"55-59",9:"60-64",10:"65-69",11:"70-74",12:"75-79",13:"80+",14:"Refused"},
    "BMICAT": {1:"Underweight",2:"Normal",3:"Overweight",4:"Obese"},
    "EDUCATION_LEVEL": {1:"Dropout",2:"High School Graduate",3:"Undergraduate",4:"Graduate",9:"Not sure"},
    "INCOME": {1:"< $15000",2:"$15000 - $25000",3:"$25000 - $35000",4:"$35000 - $50000",5:"> $50000",9:"Not sure"}
}

In [21]:
def ensure_xpt(year, retries: int = 3, timeout: int = 30):
    """
    Ensure the LLCP{year}.XPT file exists locally.

    Behavior:
    - If the ZIP (LLCP{year}.zip) is not present in DATA_DIR, download it from BASE_URL.
    - Extract the first .xpt file found in the ZIP and move it to DATA_DIR/LLCP{year}.XPT.
    - If the .xpt is already present, do nothing (cached).
    - Uses exponential backoff between download retries.

    Args:
        year (int): Year to fetch (e.g., 2019).
        retries (int): Number of download attempts (default 3).
        timeout (int): Request timeout in seconds for each attempt (default 30).

    Returns:
        str: Full path to the local .XPT file.

    Raises:
        RuntimeError: If the ZIP contains no .xpt or the zip file is bad/corrupt.
        requests.exceptions.RequestException: If download fails after retries (propagated).
    """
    DATA_PARENT_DIR = os.path.join(DATA_DIR, f"{year}")
    if not os.path.exists(DATA_PARENT_DIR):
        os.makedirs(DATA_PARENT_DIR, exist_ok=True)
    zip_path = os.path.join(DATA_PARENT_DIR, f"LLCP{year}.zip")
    xpt_path = os.path.join(DATA_PARENT_DIR, f"LLCP{year}.XPT")

    if not os.path.exists(zip_path):
        print(f"[DOWNLOAD] {year}")
        url = BASE_URL.format(year=year)
        for attempt in range(1, retries + 1):
            try:
                with requests.get(url, stream=True, timeout=timeout) as r:
                    r.raise_for_status()
                    with open(zip_path, "wb") as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            if chunk:
                                f.write(chunk)
                break
            except Exception as e:
                print(f"Download attempt {attempt} failed for {year}: {e}")
                if attempt < retries:
                    time.sleep(2 ** attempt)
                else:
                    raise
    else:
        print(f"[CACHED] {year} zip")

    if not os.path.exists(xpt_path):
        print(f"[UNZIP] {year}")
        try:
            with zipfile.ZipFile(zip_path, "r") as z:
                members = [m for m in z.namelist() if m.lower().endswith(".xpt")]
                if not members:
                    raise RuntimeError(f"No .xpt found in zip for {year}")
                # extract first .xpt and move it to expected path
                member = members[0]
                z.extract(member, DATA_PARENT_DIR)
                extracted = os.path.join(DATA_PARENT_DIR, member)
                # ensure destination dir exists
                os.makedirs(os.path.dirname(xpt_path), exist_ok=True)
                try:
                    shutil.move(extracted, xpt_path)
                except Exception:
                    os.replace(extracted, xpt_path)
        except zipfile.BadZipFile as e:
            raise RuntimeError(f"Bad zip file for {year}: {e}")
    else:
        print(f"[CACHED] {year} xpt")

    return xpt_path

In [7]:
def decode_value(canonical: str, year: int, val) -> Optional[str]:
    """
    Decode a single scalar value to its human-readable label.

    Parameters
    ----------
    canonical : str
        Canonical field name (key in VALUE_MAP / VALUE_TEXT_MAP).
    year : int
        Year to use for per-year mapping.
    val : scalar
        Value to decode (int, str convertible to int, or missing).

    Returns
    -------
    str or pd.NA
        Mapped label string, or pd.NA if the value is missing/unmapped.

    Notes
    -----
    - Missing values (pd.isna) or non-integer-convertible values return pd.NA.
    - First attempts to use the per-year mapping in VALUE_MAP; if not found,
      falls back to the constant mapping in VALUE_TEXT_MAP.
    """
    if pd.isna(val):
        return pd.NA
    try:
        key = int(val)
    except (ValueError, TypeError):
        return pd.NA

    # Try per-year mapping first
    per_year_map = VALUE_MAP.get(canonical, {}).get(year, {})
    label = per_year_map.get(key)
    if label is not None:
        return label

    # Fallback to constant mapping
    label = VALUE_TEXT_MAP.get(canonical, {}).get(key)
    return label if label is not None else pd.NA

In [8]:
def decode_series(canonical: str, year: int, series: pd.Series) -> pd.Series:
    """
    Vectorized decoding for a pandas Series of categorical codes.

    Behavior
    - Uses the per-year mapping in VALUE_MAP[canonical][year] if present.
      Missing keys fall back to the constant mapping VALUE_TEXT_MAP[canonical].
    - Non-numeric or non-convertible values are treated as missing and mapped to pd.NA.
    - Preserves the input index and returns a Series of labels (strings) or pd.NA.
    - Returns an empty Series with the same index if `series` is empty or None.

    Parameters
    - canonical (str): canonical variable name (key into VALUE_MAP / VALUE_TEXT_MAP)
    - year (int): year to select per-year mapping
    - series (pd.Series): input series of codes (ints or strings convertible to ints)

    Returns
    - pd.Series: decoded labels or pd.NA
    """
    if series is None:
        return pd.Series(dtype="object")
    if series.empty:
        return pd.Series(index=series.index, dtype="object")

    # coerce to integer codes where possible (nullable Int64)
    codes = pd.to_numeric(series, errors="coerce").astype("Int64")

    per_year_map = VALUE_MAP.get(canonical, {}).get(year, {})
    fallback_map = VALUE_TEXT_MAP.get(canonical, {})
    # fallback_map provides defaults, per_year_map overrides them
    combined_map = {**fallback_map, **per_year_map}

    mapped = codes.map(combined_map)

    # preserve explicit missing codes and ensure unmapped numeric codes become pd.NA
    mapped = mapped.where(~codes.isna(), pd.NA)
    mapped = mapped.where(mapped.notna(), pd.NA)

    return mapped.astype("object")

In [9]:
def normalize_days(series: pd.Series) -> pd.Series:
    """Normalize day-count style fields to the 2016 coding:
    1 -> Zero days, 2 -> 1-13 days, 3 -> 14+ days, 9 -> Refused

    Rules:
    - 88 or 0 => 1 (Zero days)
    - 1-13 => 2 (1-13 days)
    - >=14 and < 97 => 3 (14+ days)
    - 9 or 99 => 9 (Refused)
    - everything else => pd.NA
    """
    if series is None or series.empty:
        return pd.Series(dtype="Int64")
    nums = pd.to_numeric(series, errors="coerce")

    def _norm(v):
        if pd.isna(v):
            return pd.NA
        vi = int(v)
        if vi in (9, 99):
            return 9
        if vi in (88, 0):
            return 1
        if 1 <= vi <= 13:
            return 2
        if vi >= 14 and vi < 97:
            return 3
        return pd.NA

    normalized = nums.apply(lambda x: _norm(x)).astype("Int64")
    return normalized

In [10]:
def validate_year_mappings(df_raw: pd.DataFrame, year: int) -> list:
    """Return list of canonical fields that have no matching columns in df_raw and log a summary.
    Parameters
    ----------
    df_raw : pd.DataFrame
        Raw dataframe loaded from BRFSS XPT for the given year.
    year : int
        Year of the dataframe to validate.
    Returns
    -------
    list
        List of canonical field names missing from df_raw columns.
    Notes
    -----
    - Uses VAR_MAP to determine expected columns for the given year.
    - Prints a summary of missing fields.
    """
    missing = []
    for canonical, year_fields in VAR_MAP.items():
        candidates = None
        if isinstance(year_fields, dict):
            candidates = year_fields.get(year)
        if candidates is None:
            candidates = []
        elif isinstance(candidates, str):
            candidates = [candidates]
        elif not isinstance(candidates, (list, tuple, set)):
            try:
                candidates = list(candidates)
            except Exception:
                candidates = []

        found = any(c in df_raw.columns for c in candidates)
        if not found:
            missing.append(canonical)

    print(f"Year {year}: {len(missing)} of {len(VAR_MAP)} canonical fields missing")
    if missing:
        print(f"Missing fields for {year}: {missing}")
    return missing

In [11]:
def load_year(year):
    """Load and normalize BRFSS data for a given year.
    Parameters
    ----------
    year : int
        Year to load (e.g., 2019).
    Returns
    -------
    pd.DataFrame
        Normalized dataframe with canonical columns and YEAR column added.
    Notes
    -----
    - Loads the SAS XPT file for the given year.
    - Maps raw columns to canonical columns using VAR_MAP.
    - Normalizes day-count health fields to 2016 enum codes.
    - Logs missing columns and normalization issues.
   """
    xpt = ensure_xpt(year)
    try:
        df_raw = pd.read_sas(xpt, format="xport", encoding="latin1")
    except Exception as e:
        print(f"Failed to read SAS XPT for year {year}: {e}")
        return pd.DataFrame()

    # validate mappings and log a summary
    validate_year_mappings(df_raw, year)

    out = pd.DataFrame()

    for canonical, year_fields in VAR_MAP.items():
        # normalize candidate(s) to a list
        candidates = None
        if isinstance(year_fields, dict):
            candidates = year_fields.get(year)
        if candidates is None:
            candidates = []
        elif isinstance(candidates, str):
            candidates = [candidates]
        elif not isinstance(candidates, (list, tuple, set)):
            try:
                candidates = list(candidates)
            except Exception:
                candidates = []

        raw = None
        for candidate in candidates:
            if candidate in df_raw.columns:
                raw = df_raw[candidate]
                break

        if raw is None:
            print(f"Year {year}: no column found for {canonical} among {candidates}")
            out[f"{canonical}"] = np.nan
        else:
            # Normalize day-count health fields to 2016 enum codes
            if canonical in ("PHYSICAL_HEALTH_STATUS", "MENTAL_HEALTH_STATUS"):
                try:
                    normalized = normalize_days(raw)
                    out[f"{canonical}"] = normalized
                except Exception as e:
                    print(f"Normalization failed for {canonical} in {year}: {e}")
                    out[f"{canonical}"] = raw
            else:
                out[f"{canonical}"] = raw

    out["YEAR"] = year
    return out

In [12]:
def _get_memory_bytes() -> int:
    """
    Return memory usage in bytes. Prefer psutil if available, otherwise fall back to tracemalloc current traced memory.
    Returns
    -------
    int
        Memory usage in bytes.
    """
    try:
        import psutil
        proc = psutil.Process(os.getpid())
        return int(proc.memory_info().rss)
    except Exception:
        # Ensure tracemalloc is started
        if not tracemalloc.is_tracing():
            tracemalloc.start()
        current, peak = tracemalloc.get_traced_memory()
        return int(current)

In [13]:
def _format_bytes(b: int) -> str:
    """
    Format bytes as human-readable string (e.g., 1.23 MB).
    Parameters
    ----------
    b : int
        Number of bytes.
    Returns
    -------
    str
        Formatted string with appropriate unit.
    """
    if b is None:
        return "0 B"
    sign = "" if b >= 0 else "-"
    b = abs(int(b))
    for unit in ["B", "KB", "MB", "GB", "TB"]:
        if b < 1024:
            return f"{sign}{b:.2f} {unit}"
        b /= 1024
    return f"{sign}{b:.2f} PB"

In [14]:
def load_multi_year(years: list, output_dir: str = "data_processed", merged_output_filename: str = "BRFSS.csv") -> str:
    # Use a stable canonical column order derived from VAR_MAP to ensure CSV consistency
    canonical_cols = list(VAR_MAP.keys()) + ["YEAR"]
    first = True
    total_rows = 0
    for year in years:
        mem_before = _get_memory_bytes()
        print(f"Year {year}: memory before processing: {_format_bytes(mem_before)}")

        df_year = load_year(year)
        if df_year.empty:
            print(f"Year {year}: no data loaded; skipping write")
            mem_after = _get_memory_bytes()
            print(f"Year {year}: memory after processing (skipped): {_format_bytes(mem_after)} delta: {_format_bytes(mem_after - mem_before)}")
            continue

        # ensure columns are in canonical order and any missing columns are added as NA
        df_year = df_year.reindex(columns=canonical_cols)
        # Make the YEAR column the first column
        cols = df_year.columns.tolist()
        cols.insert(0, cols.pop(cols.index("YEAR")))
        df_year = df_year[cols]

        # Resolve enum codes to text labels using VALUE_TEXT_MAP before writing
        # This maps numeric/encoded values to human-readable labels and preserves missing values.
        # for col in list(VAR_MAP.keys()):
        #     if col not in df_year.columns:
        #         continue
        #     mapping = VALUE_TEXT_MAP.get(col)
        #     if not mapping:
        #         continue
        #     try:
        #         codes = pd.to_numeric(df_year[col], errors="coerce")
        #         mapped = codes.map(mapping).astype("object")
        #         # preserve missing / NaN as pd.NA
        #         mapped = mapped.where(~codes.isna(), pd.NA)
        #         df_year[col] = mapped
        #     except Exception as e:
        #         print(f"Failed to map enum values for {col} in year {y}: {e}")

        print(f"Year {year}: writing {df_year.shape[0]} rows to CSV {os.path.join(output_dir, f'brfss_{year}.csv')}")
        df_year.to_csv(os.path.join(output_dir, f"brfss_{year}.csv"), mode="w", header=first, index=False)

        print(f"Year {year}: appending {df_year.shape[0]} rows to merged CSV {os.path.join(output_dir, merged_output_filename)}")
        df_year.to_csv(os.path.join(output_dir, merged_output_filename), mode="w" if first else "a", header=first, index=False)
        
        total_rows += df_year.shape[0]
        print(f"[WROTE] Year {year}: {df_year.shape[0]} rows")

        # attempt to free memory and measure after write
        del df_year
        gc.collect()
        mem_after = _get_memory_bytes()
        print(f"Year {year}: memory after processing: {_format_bytes(mem_after)} delta: {_format_bytes(mem_after - mem_before)}")

        first = False

    print(f"[DONE] {total_rows} rows written to {merged_output_filename}")
    return merged_output_filename

In [22]:
start_year = 2015
end_year = 2024
merged_output_file_name = f"BRFSS_{start_year}_{end_year}.csv"
years = list(range(start_year, end_year))
output_dir = PROCESSED_DATA_DIR
outputfile = os.path.join(output_dir, merged_output_file_name)

load_multi_year(years, output_dir, merged_output_file_name)

Year 2015: memory before processing: 140.97 MB
[CACHED] 2015 zip
[UNZIP] 2015


RuntimeError: No .xpt found in zip for 2015